<a href="https://colab.research.google.com/github/davidlealo/100profes/blob/master/multiple_models_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Uso de múltiples modelos de AI



#### Paso 1: Instalación de librerías

In [1]:
!pip install gradio requests -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.2 MB/s eta 0:00:00


#### Paso 2: Configuración inicial

In [2]:
import os
import requests

# Variable global para almacenar la API key del usuario
user_api_key = None

#### Paso 3: Función para interactuar con Mistral


In [3]:
def mistral_request(prompt, api_key):
    """
    Envía un prompt a la API de Mistral y devuelve la respuesta.
    """
    url = "https://api.mistral.ai/v1/completions"  # Cambia si usas otro endpoint
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "mistral",  # O el modelo que uses
        "prompt": prompt,
        "max_tokens": 100
    }
    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        return response.json().get("choices")[0].get("text", "No response.")
    else:
        return f"Error: {response.status_code} - {response.text}"

In [ ]:


# Paso 4: Identificar intención y asignar acción
def identify_and_call_model(prompt, api_key):
    """
    Detecta la intención del usuario y realiza la acción correspondiente.
    """
    if "análisis de sentimientos" in prompt.lower():
        return mistral_request("Realiza un análisis de sentimientos: " + prompt, api_key)
    elif "traducción" in prompt.lower():
        return mistral_request("Traduce lo siguiente: " + prompt, api_key)
    else:
        return mistral_request(prompt, api_key)

# Paso 5: Interfaz con Gradio
import gradio as gr

def set_api_key(api_key):
    """
    Guarda la API key proporcionada por el usuario.
    """
    global user_api_key
    user_api_key = api_key
    return "API Key configurada correctamente."

def main_function(user_input):
    """
    Procesa la entrada del usuario y llama al modelo adecuado.
    """
    if not user_api_key:
        return "Por favor, configura tu API Key primero."

    try:
        response = identify_and_call_model(user_input, user_api_key)
        return f"Respuesta de la AI:\n{response}"
    except Exception as e:
        return f"Error procesando tu solicitud: {str(e)}"

# Crear la interfaz de Gradio
api_key_input = gr.Textbox(label="Introduce tu API Key", type="password", placeholder="Tu API Key aquí")
set_key_button = gr.Button("Configurar API Key")
chat_input = gr.Textbox(label="Prompt", placeholder="Escribe tu solicitud aquí...")
chat_output = gr.Textbox(label="Respuesta", lines=10)

# Interfaz con dos funciones: configurar API Key y procesar el chat
interface = gr.Interface(
    fn=[set_api_key, main_function],
    inputs=[api_key_input, chat_input],
    outputs=chat_output,
    live=True,
    title="Chat con Mistral",
    description="Un chatbot que entiende tus necesidades y usa modelos de Mistral."
)

# Configuración de botones
interface.launch()
